In [47]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.sequence import pad_sequences

import os

In [48]:
imdb_train, ds_info = tfds.load(name="imdb_reviews",
                                split="train",
                                with_info=True, as_supervised=True)

imdb_test = tfds.load(name="imdb_reviews", split="test", as_supervised=True)

INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load dataset info from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
INFO:absl:Reusing dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split train, from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load dataset info from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
INFO:absl:Reusing dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split test, from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0


### Keras Tokenizer

In [ ]:
X_train = list(map(lambda x: x[0].numpy().decode('utf-8'), imdb_train))
y_train = list(map(lambda x: x[1], imdb_train))

In [ ]:
%%time
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                                  lower=True,
                                                  split=" ",)
tokenizer.fit_on_texts(X_train)

X_encoded = tokenizer.texts_to_sequences(X_train)
X_encoded = pad_sequences(X_encoded, padding="post", maxlen=150) 

Wall time: 9.88 s


### TFDS Tokenizer

In [ ]:
tokenizer_tfds = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()
MAX_TOKENS = 0
for example, label in imdb_train:
    some_tokens = tokenizer_tfds.tokenize(example.numpy())
    if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
    vocabulary_set.update(some_tokens)
    
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, lowercase=True, tokenizer=tokenizer_tfds)

vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKENS)

93931 2525


In [ ]:
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post',maxlen=150)
    return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
    
    encoded = tf.py_function(encode_pad_transform,
                             inp=[sample],
                             Tout=(tf.int64))
    
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label 

In [ ]:
encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

### Loading pre-trained GloVe embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
from google.colab import drive

drive.mount('drive')
glove_path = 'drive/MyDrive/GloVe/glove.6B.50d.txt'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
dict_w2v = {}
with open(glove_path, "rb") as file:
    for line in file:
        tokens = line.split()
        word = tokens[0].decode('utf-8')
        vector = np.array(tokens[1:], dtype=np.float32)
        
        if vector.shape[0] == 50:
            dict_w2v[word] = vector
        else:
            print("There was an issue with " + word)
            
# let's check the vocabulary size
print("Dictionary Size: ", len(dict_w2v))

Dictionary Size:  400000


In [ ]:
vocab_size = imdb_encoder.vocab_size
embedding_dim = 50
embedding_matrix = np.zeros((vocab_size, embedding_dim))

### Tfds

In [ ]:
unk_cnt = 0
unk_set = set()
for word in imdb_encoder.tokens:
    embedding_vector = dict_w2v.get(word)
    
    if embedding_vector is not None:
        tkn_id = imdb_encoder.encode(word)[0]
        embedding_matrix[tkn_id] = embedding_vector
    else:
        unk_cnt += 1
        unk_set.add(word)
# Print how many weren't found
print("Total unknown words: ", unk_cnt)

Total unknown words:  14553


### Keras

In [ ]:
unk_cnt = 0
unk_set = set()
for word in list(tokenizer.word_index.keys()):
    embedding_vector = dict_w2v.get(word)
    
    if embedding_vector is not None:
        tkn_id = tokenizer.word_index[word]
        embedding_matrix[tkn_id] = embedding_vector
    else:
        unk_cnt += 1
        unk_set.add(word)
# Print how many weren't found
print("Total unknown words: ", unk_cnt)

Total unknown words:  28423


### Build model

In [ ]:
vocab_size = imdb_encoder.vocab_size # len(chars)
# Number of RNN units
rnn_units = 64
#batch size
BATCH_SIZE=100

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size, train_emb=False):
    model = tf.keras.Sequential([
        Embedding(vocab_size, embedding_dim, mask_zero=True,
                  weights=[embedding_matrix], trainable=train_emb),
        Bidirectional(LSTM(rnn_units, return_sequences=True,
                           dropout=0.5)),
        Bidirectional(LSTM(rnn_units, dropout=0.25)),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
model_fe = build_model_bilstm(
    vocab_size = vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model_fe.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 50)          4696550   
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 128)         58880     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 4,854,375
Trainable params: 157,825
Non-trainable params: 4,696,550
_________________________________________________________________


In [ ]:
model_fe.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])

In [ ]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)
model_fe.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 33s 82ms/step - loss: 0.6347 - accuracy: 0.6282 - precision: 0.6288 - recall: 0.6348
Epoch 2/10
250/250 [==============================] - 22s 90ms/step - loss: 0.5586 - accuracy: 0.7157 - precision: 0.7223 - recall: 0.7025
Epoch 3/10
250/250 [==============================] - 23s 90ms/step - loss: 0.5194 - accuracy: 0.7432 - precision: 0.7470 - recall: 0.7379
Epoch 4/10
250/250 [==============================] - 22s 90ms/step - loss: 0.4896 - accuracy: 0.7647 - precision: 0.7644 - recall: 0.7665
Epoch 5/10
250/250 [==============================] - 23s 90ms/step - loss: 0.5145 - accuracy: 0.7462 - precision: 0.7491 - recall: 0.7424
Epoch 6/10
250/250 [==============================] - 22s 90ms/step - loss: 0.4608 - accuracy: 0.7800 - precision: 0.7829 - recall: 0.7757
Epoch 7/10
250/250 [==============================] - 23s 91ms/step - loss: 0.4534 - accuracy: 0.7883 - precision: 0.7968 - recall: 0.7752
Epoch 8/10
250/250 [=======

### Fine-tuning

In [ ]:
model_ft = build_model_bilstm(
 vocab_size=vocab_size,
 embedding_dim=embedding_dim,
 rnn_units=rnn_units,
 batch_size=BATCH_SIZE,
 train_emb=True)
model_ft.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 50)          4696550   
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 128)         58880     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 4,854,375
Trainable params: 4,854,375
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_ft.compile(loss='binary_crossentropy',
 optimizer='adam',
 metrics=['accuracy', 'Precision', 'Recall'])
model_ft.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 44s 125ms/step - loss: 0.6129 - accuracy: 0.6483 - precision: 0.6497 - recall: 0.6520
Epoch 2/10
250/250 [==============================] - 33s 131ms/step - loss: 0.4159 - accuracy: 0.8108 - precision: 0.8106 - recall: 0.8120
Epoch 3/10
250/250 [==============================] - 33s 131ms/step - loss: 0.3367 - accuracy: 0.8522 - precision: 0.8549 - recall: 0.8491
Epoch 4/10
250/250 [==============================] - 33s 130ms/step - loss: 0.2909 - accuracy: 0.8764 - precision: 0.8749 - recall: 0.8789
Epoch 5/10
250/250 [==============================] - 33s 131ms/step - loss: 0.2436 - accuracy: 0.9008 - precision: 0.9008 - recall: 0.9011
Epoch 6/10
250/250 [==============================] - 33s 131ms/step - loss: 0.2209 - accuracy: 0.9113 - precision: 0.9097 - recall: 0.9135
Epoch 7/10
250/250 [==============================] - 33s 131ms/step - loss: 0.1858 - accuracy: 0.9259 - precision: 0.9270 - recall: 0.9250
Epoch 8/10
250/250 [

### BERT

In [49]:
!pip install transformers==3.0.2

In [50]:
from transformers import BertTokenizer
bert_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_name,
                                          add_special_tokens=True,
                                          do_lower_case=False,
                                          max_length=150,
                                          pad_to_max_length=True)

In [51]:
def bert_encoder(review):
 txt = review.numpy().decode('utf-8')
 encoded = tokenizer.encode_plus(txt, add_special_tokens=True,
                                 max_length=150,
                                 pad_to_max_length=True,
                                 return_attention_mask=True,
                                 truncation=True,
                                 return_token_type_ids=True)
 
 return encoded['input_ids'], encoded['token_type_ids'], encoded['attention_mask']

In [52]:
bert_train = [bert_encoder(r) for r, l in imdb_train]
bert_lbl = [l for r, l in imdb_train]
bert_train = np.array(bert_train)
bert_lbl = tf.keras.utils.to_categorical(bert_lbl, num_classes=2)

In [55]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(bert_train,
                                                  bert_lbl,
                                                  test_size=0.2,
                                                  random_state=42)

print(x_train.shape, y_train.shape)

(20000, 3, 150) (20000, 2)


In [56]:
tr_reviews, tr_segments, tr_masks = np.split(x_train, 3, axis=1)
val_reviews, val_segments, val_masks = np.split(x_val, 3, axis=1)

tr_reviews = tr_reviews.squeeze()
tr_segments = tr_segments.squeeze()
tr_masks = tr_masks.squeeze()

val_reviews = val_reviews.squeeze()
val_segments = val_segments.squeeze()
val_masks = val_masks.squeeze()

In [57]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
 return {"input_ids": input_ids,
         "attention_mask": attention_masks,
         "token_type_ids": token_type_ids},y

In [58]:
train_ds = tf.data.Dataset.from_tensor_slices((tr_reviews,
                                               tr_masks, tr_segments, y_train)).map(example_to_features).shuffle(100).batch(16)

valid_ds = tf.data.Dataset.from_tensor_slices((val_reviews,
                                               val_masks, val_segments, y_val)).map(example_to_features).shuffle(100).batch(16)

In [59]:
from transformers import TFBertForSequenceClassification

bert_model = TFBertForSequenceClassification.from_pretrained(bert_name)

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 5)
        name: serving_default_input_ids:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0521 11:32:38.297619 140153549420416 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/resour

In [61]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bert_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
bert_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [62]:
bert_history = bert_model.fit(train_ds, epochs=3, validation_data=valid_ds)

Epoch 1/3
1250/1250 [==============================] - 702s 549ms/step - loss: 0.4206 - accuracy: 0.7980 - val_loss: 0.2863 - val_accuracy: 0.8744
Epoch 2/3
1250/1250 [==============================] - 689s 551ms/step - loss: 0.2118 - accuracy: 0.9169 - val_loss: 0.2838 - val_accuracy: 0.8854
Epoch 3/3
1250/1250 [==============================] - 688s 550ms/step - loss: 0.1073 - accuracy: 0.9645 - val_loss: 0.3738 - val_accuracy: 0.8860


In [63]:
bert_test = [bert_encoder(r) for r,l in imdb_test]
bert_tst_lbl = [l for r, l in imdb_test]
bert_test2 = np.array(bert_test)

bert_tst_lbl2 = tf.keras.utils.to_categorical (bert_tst_lbl,num_classes=2)

ts_reviews, ts_segments, ts_masks = np.split(bert_test2, 3, axis=1)

ts_reviews = ts_reviews.squeeze()
ts_segments = ts_segments.squeeze()
ts_masks = ts_masks.squeeze()

test_ds = tf.data.Dataset.from_tensor_slices((ts_reviews,
                                              ts_masks, ts_segments, bert_tst_lbl2)).map(example_to_features).shuffle(100).batch(16)


In [83]:
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    bert_model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1



INFO:tensorflow:Assets written to: /tmp/1/assets


INFO:tensorflow:Assets written to: /tmp/1/assets



Saved model:
total 10060
drwxr-xr-x 2 root root     4096 May 21 11:28 assets
-rw-r--r-- 1 root root 10289460 May 21 11:28 saved_model.pb
drwxr-xr-x 2 root root     4096 May 21 11:28 variables


### Custom model with BERT

In [66]:
from transformers import TFBertModel
bert_name = 'bert-base-cased'
bert = TFBertModel.from_pretrained(bert_name)
bert.summary()

- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [68]:
max_seq_len = 150
inp_ids = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="input_ids")
att_mask = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="attention_mask")
seg_ids = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="token_type_ids")

In [77]:
inp_dict = {"input_ids": inp_ids,
 "attention_mask": att_mask,
 "token_type_ids": seg_ids}
outputs = bert(inp_dict)

'''
  The first output has embeddings for each of the input tokens including the special
  tokens [CLS] and [SEP]. The second output corresponds to the output of the [CLS]
  token. This output will be used further in the mode
'''

bert.trainable = False # don't train BERT any more

In [78]:
x = tf.keras.layers.Dropout(0.2)(outputs[1])
x = tf.keras.layers.Dense(200, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(2, activation='sigmoid')(x)

custom_model = tf.keras.models.Model(inputs=inp_dict, outputs=x)

In [79]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
custom_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [80]:
custom_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
attention_mask (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 150, 768), ( 108310272   attention_mask[0][0]             
                                                                 input_ids[0][0]            

In [81]:
custom_history = custom_model.fit(train_ds, epochs=3, validation_data=valid_ds)

Epoch 1/3
1250/1250 [==============================] - 304s 234ms/step - loss: 0.4274 - accuracy: 0.8135 - val_loss: 0.3646 - val_accuracy: 0.8358
Epoch 2/3
1250/1250 [==============================] - 290s 232ms/step - loss: 0.3703 - accuracy: 0.8407 - val_loss: 0.3582 - val_accuracy: 0.8376
Epoch 3/3
1250/1250 [==============================] - 291s 232ms/step - loss: 0.3632 - accuracy: 0.8410 - val_loss: 0.3549 - val_accuracy: 0.8378


In [ ]:
custom_model.evaluate(test_ds)